# Implementation of SCD Type 1, 2, 3, and 4 in Python


## Importing Libraries and Data Preparation
In this step, we import necessary libraries like pandas and datetime. We then create two datasets: 
- The "old" dataset representing the original data.
- The "new" dataset representing the updated data with changes. 
These datasets will be used to demonstrate different types of Slowly Changing Dimensions (SCD).


In [1]:
# Import necessary libraries
import pandas as pd
from datetime import datetime

# Old dataset
old_data = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['John Doe', 'Jane Smith', 'Mary Johnson'],
    'city': ['New York', 'Los Angeles', 'Chicago'],
    'salary': [70000, 80000, 120000]
})

# New dataset
new_data = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['John Doe', 'Jane Doe', 'Mary Johnson'],
    'city': ['New York', 'San Francisco', 'Chicago'],
    'salary': [75000, 85000, 120000]
})

# Display the initial datasets
print("Old Data:")
display(old_data)
print("\nNew Data:")
display(new_data)


Old Data:


,id,name,city,salary
0,1,John Doe,New York,70000
1,2,Jane Smith,Los Angeles,80000
2,3,Mary Johnson,Chicago,120000



New Data:


,id,name,city,salary
0,1,John Doe,New York,75000
1,2,Jane Doe,San Francisco,85000
2,3,Mary Johnson,Chicago,120000


## SCD Type 1 - Overwrite
SCD Type 1 simply overwrites the existing data with new values. This means any previous values are lost, and only the most recent information is retained. 
We loop through the new data and update the corresponding records in the old data where changes have occurred.


In [2]:
# SCD Type 1: Overwrite
scd_type1 = old_data.copy()

# Updating rows where changes occurred
for index, new_row in new_data.iterrows():
    scd_type1.loc[scd_type1['id'] == new_row['id'], ['name', 'city', 'salary']] = new_row[['name', 'city', 'salary']]

print("\nSCD Type 1 (Overwrite):")
display(scd_type1)



SCD Type 1 (Overwrite):


C:\Users\hamza\AppData\Local\Temp\ipykernel_51912\1003437780.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  scd_type1.loc[scd_type1['id'] == new_row['id'], ['name', 'city', 'salary']] = new_row[['name', 'city', 'salary']]


,id,name,city,salary
0,1,NaN,NaN,NaN
1,2,NaN,NaN,NaN
2,3,NaN,NaN,NaN


## SCD Type 2 - Add New Row (Preserve History)
In SCD Type 2, we retain a history of changes by adding new rows for each change. Each row contains start and end dates to indicate the period of validity, along with a column indicating whether the record is currently active (`is_current`). 
This approach ensures that we maintain a complete history of all changes made over time.


In [3]:
# Adding effective and expiry dates to track the history
current_date = datetime.now().strftime('%Y-%m-%d')

# Adding necessary columns for SCD Type 2
old_data_scd2 = old_data.copy()
old_data_scd2['is_current'] = 'Y'
old_data_scd2['effective_date'] = current_date
old_data_scd2['expiry_date'] = None

new_rows = []

for index, new_row in new_data.iterrows():
    # Find matching rows in the old data
    match = old_data_scd2[(old_data_scd2['id'] == new_row['id']) & (old_data_scd2['is_current'] == 'Y')]
    
    if not match.empty and not match.iloc[0].equals(new_row):
        # Update the old record to be no longer current
        old_data_scd2.loc[old_data_scd2['id'] == new_row['id'], 'is_current'] = 'N'
        old_data_scd2.loc[old_data_scd2['id'] == new_row['id'], 'expiry_date'] = current_date
        
        # Insert the new row as the current version
        new_row['is_current'] = 'Y'
        new_row['effective_date'] = current_date
        new_row['expiry_date'] = None
        new_rows.append(new_row)

# Append new rows to old data
scd_type2 = pd.concat([old_data_scd2, pd.DataFrame(new_rows)], ignore_index=True)

print("\nSCD Type 2 (Add New Row):")
display(scd_type2)



SCD Type 2 (Add New Row):


,id,name,city,salary,is_current,effective_date,expiry_date
0,1,John Doe,New York,70000,N,2024-09-23,2024-09-23
1,2,Jane Smith,Los Angeles,80000,N,2024-09-23,2024-09-23
2,3,Mary Johnson,Chicago,120000,N,2024-09-23,2024-09-23
3,1,John Doe,New York,75000,Y,2024-09-23,None
4,2,Jane Doe,San Francisco,85000,Y,2024-09-23,None
5,3,Mary Johnson,Chicago,120000,Y,2024-09-23,None


## SCD Type 3 - Add New Column
SCD Type 3 adds a new column to store the previous value of the changed attribute. This approach captures the immediate past value while maintaining the current value. 
It allows us to track the most recent change but does not retain the full history of changes.


In [5]:
# Copying old data for SCD Type 3
scd_type3 = old_data.copy()
scd_type3['previous_city'] = scd_type3['city']

# Updating with the new values
for index, new_row in new_data.iterrows():
    match = scd_type3[scd_type3['id'] == new_row['id']]
    if not match.empty:
        scd_type3.loc[scd_type3['id'] == new_row['id'], ['name', 'city', 'salary']] = new_row[['name', 'city', 'salary']]

print("\nSCD Type 3 (Add New Column):")
display(scd_type3)


SCD Type 3 (Add New Column):


C:\Users\hamza\AppData\Local\Temp\ipykernel_51912\2438326019.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  scd_type3.loc[scd_type3['id'] == new_row['id'], ['name', 'city', 'salary']] = new_row[['name', 'city', 'salary']]


,id,name,city,salary,previous_city
0,1,NaN,NaN,NaN,New York
1,2,NaN,NaN,NaN,Los Angeles
2,3,NaN,NaN,NaN,Chicago


## SCD Type 4 - Using a History Table
In SCD Type 4, we maintain a separate history table that stores all historical changes. The main table contains only the current data. 
This method allows us to keep a detailed history of changes without modifying the structure of the main table, making it an efficient way to track changes over time.


In [6]:
# Main table will contain the latest data
scd_type4_main = new_data.copy()

# History table will contain all previous records
scd_type4_history = old_data.copy()
scd_type4_history['record_version'] = 1  # Add version tracking

# Version tracking for new data changes
for index, new_row in new_data.iterrows():
    match = old_data[old_data['id'] == new_row['id']]
    if not match.empty and not match.iloc[0].equals(new_row):
        match['record_version'] = 1
        scd_type4_history = pd.concat([scd_type4_history, match], ignore_index=True)

print("\nSCD Type 4 (Using History Table):")
print("Main Table:")
display(scd_type4_main)
print("\nHistory Table:")
display(scd_type4_history)



SCD Type 4 (Using History Table):
Main Table:


C:\Users\hamza\AppData\Local\Temp\ipykernel_51912\256994068.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match['record_version'] = 1
C:\Users\hamza\AppData\Local\Temp\ipykernel_51912\256994068.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match['record_version'] = 1


,id,name,city,salary
0,1,John Doe,New York,75000
1,2,Jane Doe,San Francisco,85000
2,3,Mary Johnson,Chicago,120000



History Table:


,id,name,city,salary,record_version
0,1,John Doe,New York,70000,1
1,2,Jane Smith,Los Angeles,80000,1
2,3,Mary Johnson,Chicago,120000,1
3,1,John Doe,New York,70000,1
4,2,Jane Smith,Los Angeles,80000,1
